In [1]:
from pydrake.systems.framework import DiagramBuilder
from pydrake.systems.analysis import Simulator
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph, MultibodyPlant
from pydrake.multibody.parsing import Parser
from pydrake.geometry import Meshcat, MeshcatVisualizerCpp
from pydrake.common.eigen_geometry import Quaternion, AngleAxis
from pydrake.math import RigidTransform, RotationMatrix
from pydrake.trajectories import PiecewisePose, PiecewisePolynomial
from pydrake.systems.primitives import TrajectorySource, Multiplexer, ConstantVectorSource

import numpy as np

from crane_x7_manipulation import crane_x7_station, end_effector_trajectory, meshcat_utils


URDF_PATH = 'rsc/crane_x7_description/crane_x7.urdf'

In [2]:
meshcat = meshcat_utils.start_meshcat(True)

[2021-10-20 21:36:18.078] [console] [info] Meshcat listening for connections at http://localhost:7000


<IPython.core.display.Javascript object>

In [3]:
def create_model(urdf_path):
    station = crane_x7_station.CraneX7Station(urdf_path)
    plant, model = station.create_crane_x7_plant()
    plant.Finalize()
    context = plant.CreateDefaultContext()

    plant.SetPositions(context, [-1.57, 0.1, 0, 0, 0, 1.6, 0, 0, 0]) # 7DOF (arm) + 2DOF (gripper)
    plant.GetJointByName("crane_x7_upper_arm_revolute_part_rotate_joint").set_angle(context, -1.2)
    plant.get_actuation_input_port().FixValue(context, np.zeros(9)) # 7DOF (arm) + 2DOF (gripper)
    simulator = Simulator(plant, context)
    simulator.AdvanceTo(5.0)

In [4]:
create_model(URDF_PATH)

In [5]:
def create_model_scene_graph(urdf_path):
    builder = DiagramBuilder()

    station = crane_x7_station.CraneX7Station(urdf_path)
    plant, model, scene_graph = station.create_crane_x7_plant_scene_graph(
        builder=builder, time_step=1.0e-03)
    plant.Finalize()

    visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    diagram.Publish(context)

    plant_context = plant.GetMyMutableContextFromRoot(context)
    plant.SetPositions(plant_context, station.get_home_position()) # 7DOF (arm) + 2DOF (gripper)
    plant.get_actuation_input_port().FixValue(plant_context, np.zeros(9)) # 7DOF (arm) + 2DOF (gripper)

    # real time simulation on 
    simulator = Simulator(diagram, context)
    simulator.set_target_realtime_rate(1.0)
    simulator.AdvanceTo(5.0)

In [6]:
create_model_scene_graph(URDF_PATH)

In [52]:
def make_gripper_frames(X_G, X_O):
    # Define the gripper pose relative to the object when in grasp.
    p_GgraspO = [0, 0.0, 0]
    R_GgraspO = RotationMatrix.MakeXRotation(np.pi/2.0).multiply(RotationMatrix.MakeZRotation(np.pi/2.0))
    X_GgraspO = RigidTransform(R_GgraspO, p_GgraspO)
    X_OGgrasp = X_GgraspO.inverse()
    # pregrasp is negative y in the gripper.
    X_GgraspGpregrasp = RigidTransform([0, -0.10, 0])

    X_G["pick"] = X_O["initial"].multiply(X_OGgrasp)
    X_G["prepick"] = X_G["pick"].multiply(X_GgraspGpregrasp)
    X_G["place"] = X_O["goal"].multiply(X_OGgrasp)
    X_G["preplace"] = X_G["place"].multiply(X_GgraspGpregrasp)

    # I'll interpolate a halfway orientation by converting to axis angle and halving the angle.
    X_GprepickGpreplace = X_G["prepick"].inverse().multiply(X_G["preplace"])
    angle_axis = X_GprepickGpreplace.rotation().ToAngleAxis()
    X_GprepickGclearance = RigidTransform(AngleAxis(angle=angle_axis.angle()/2.0, axis=angle_axis.axis()), 
                                          X_GprepickGpreplace.translation()/2.0 + np.array([0, -0.1, 0]))
    X_G["clearance"] = X_G["prepick"].multiply(X_GprepickGclearance)

    # Now let's set the timing
    times = {"initial": 0}
    X_GinitialGprepick = X_G["initial"].inverse().multiply(X_G["prepick"])
    times["prepick"] = times["initial"] + 10.0*np.linalg.norm(X_GinitialGprepick.translation())
    # Allow some time for the gripper to close.
    times["pick_start"] = times["prepick"] + 2.0
    times["pick_end"] = times["pick_start"] + 2.0
    X_G["pick_start"] = X_G["pick"]
    X_G["pick_end"] = X_G["pick"]
    times["postpick"] = times["pick_end"] + 2.0
    X_G["postpick"] = X_G["prepick"]
    time_to_from_clearance = 10.0*np.linalg.norm(X_GprepickGclearance.translation())
    times["clearance"] = times["postpick"] + time_to_from_clearance
    times["preplace"] = times["clearance"] + time_to_from_clearance
    times["place_start"] = times["preplace"] + 2.0
    times["place_end"] = times["place_start"] + 2.0
    X_G["place_start"] = X_G["place"]
    X_G["place_end"] = X_G["place"]
    times["postplace"] = times["place_end"] + 2.0
    X_G["postplace"] = X_G["preplace"]

    return X_G, times


def make_gripper_trajectory(X_G, times):
    sample_times = []
    poses = []
    for name in ["initial", "prepick", "pick_start", "pick_end", "postpick",
                 "clearance", "preplace", "place_start", "place_end",
                 "postplace"]:
        sample_times.append(times[name])
        poses.append(X_G[name])

    return PiecewisePose.MakeLinear(sample_times, poses)


def create_differential_ik_controller(urdf_path):
    builder = DiagramBuilder()

    station = crane_x7_station.CraneX7Station(urdf_path)
    plant, model, scene_graph = station.create_crane_x7_plant_scene_graph(
        builder=builder, time_step=1.0e-03)
    plant.Finalize()

    visualizer = MeshcatVisualizerCpp.AddToBuilder(builder, scene_graph, meshcat)
    diff_ik = builder.AddSystem(station.create_diff_ik())
    invdyn_controller = builder.AddSystem(station.create_invdyn_controller())
    X_O = {"initial": RigidTransform(RotationMatrix.MakeZRotation(np.pi/2.0), [0.3, 0.0, 0.08]),
           "goal": RigidTransform(RotationMatrix.MakeZRotation(np.pi), [0.2, 0.2, 0.08])}
    temp_context = plant.CreateDefaultContext()
    temp_plant_context = plant.GetMyContextFromRoot(temp_context)
    plant.SetPositions(temp_plant_context, station.get_home_position()) 
    X_G = {"initial": plant.EvalBodyPoseInWorld(temp_plant_context, plant.GetBodyByName(station.end_effector_name))}
    
    gripper_frames, times = make_gripper_frames(X_G, X_O)
    print(f"Sanity check: The entire maneuver will take {times['postplace']} seconds to execute.")

    gripper_traj = make_gripper_trajectory(gripper_frames, times)
    
    X_WE_traj = builder.AddSystem(end_effector_trajectory.EndEffectorTrajectory(station, gripper_traj))
    X_WE_traj.set_name("X_WE")

    builder.Connect(X_WE_traj.get_output_port(),
                    diff_ik.get_input_port())
    desired_state = builder.AddSystem(Multiplexer([9, 9]))
    desired_velocity = builder.AddSystem(ConstantVectorSource(np.zeros(9)))
    builder.Connect(diff_ik.get_output_port(),
                    desired_state.get_input_port(0))
    builder.Connect(desired_velocity.get_output_port(),
                    desired_state.get_input_port(1))
    builder.Connect(desired_state.get_output_port(),
                    invdyn_controller.get_input_port_desired_state())
    builder.Connect(plant.get_state_output_port(),
                    invdyn_controller.get_input_port_estimated_state())
    builder.Connect(invdyn_controller.get_output_port_control(), 
                    plant.get_actuation_input_port())
    
    diagram = builder.Build()
    context = diagram.CreateDefaultContext()
    diagram.Publish(context)
    plant_context = plant.GetMyMutableContextFromRoot(context)
    plant.SetPositions(plant_context, station.get_home_position()) 
    diff_ik_context = diff_ik.GetMyMutableContextFromRoot(context)
    diff_ik.SetPositions(diff_ik_context, station.get_home_position()) 

    simulator = Simulator(diagram, context)
    simulator.set_target_realtime_rate(1.0)
    
    simulator.AdvanceTo(20)

In [53]:
create_differential_ik_controller(URDF_PATH)

Sanity check: The entire maneuver will take 16.698858208446662 seconds to execute.
